# Poses and transforms (homogeneous coordinates)
## Pose from passive rotation

This notebook explains how passive rotations are built, how to create poses of coordinates systems, and then how to convert points from one coordinate system to another.

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vcd import utils

def plot_cs(pose_wrt_ref, name, L=1):
    # Explore the coordinate systems defined for this scene
    axis = np.array([[0, L, 0, 0, 0, 0],
            [0, 0, 0, L, 0, 0],
            [0, 0, 0, 0, 0, L],
            [1, 1, 1, 1, 1, 1]])  # matrix with several 4x1 points
    pose_wrt_ref = np.array(pose_wrt_ref).reshape(4, 4)
    axis_ref = pose_wrt_ref.dot(axis)
    origin = axis_ref[:, 0]
    x_axis_end = axis_ref[:, 1]
    y_axis_end = axis_ref[:, 3]
    z_axis_end = axis_ref[:, 5]
    ax.plot([origin[0], x_axis_end[0]], [origin[1], x_axis_end[1]], [origin[2], x_axis_end[2]], 'r-')
    ax.plot([origin[0], y_axis_end[0]], [origin[1], y_axis_end[1]], [origin[2], y_axis_end[2]], 'g-')
    ax.plot([origin[0], z_axis_end[0]], [origin[1], z_axis_end[1]], [origin[2], z_axis_end[2]], 'b-')

    ax.text(origin[0], origin[1], origin[2], r'{}'.format(name))
    ax.text(x_axis_end[0], x_axis_end[1], x_axis_end[2], 'X')
    ax.text(y_axis_end[0], y_axis_end[1], y_axis_end[2], 'Y')
    ax.text(z_axis_end[0], z_axis_end[1], z_axis_end[2], 'Z')


Poses are 4x4 matrics that encode a "passive" rotation and traslation of a coordinate system with respect to another.
For example, let's think about the Vehicle Coordinate System (VCS), aka Local Coordinate System (LCS), with x-front, y-left, z-up, centered at the rear-axle projected to the ground.

Now let's think of a camera at the front of the vehicle (FV, front-view). The camera center has a certain "position" expressed with respect to the LCS

In [ ]:
# C stands for "Center" or origin of the coordinate system, fv = front-view, wrt = with respect to, lcs = local cs
C_fv_wrt_lcs = np.array([[2.5], [0.0], [0.5]]) # 2.5 meters in front, 0.0 aligned with the vehicle axis, and 0.5 m height 

So far the camera keeps the LCS rotation, i.e. x-front, y-left, z-up

### Euler angles and passive rotations
If the camera is slightly pitched so it does focus on the road and not the sky, then, we can imagine that it rotates on the y-axis by 15º degrees, so the x-axis no longer points to the horizon but to a certain distance in the ground plane.

This rotation can be encoded with Euler angles applied in a certain order.
Right-handed rotations are defined as:

$$
Rx(\alpha) = \begin{pmatrix}1 & 0 & 0\\ 0 & cos(\alpha) & -sin(\alpha)\\ 0 & sin(\alpha) & cos(\alpha)\end{pmatrix}, Ry(\beta) = \begin{pmatrix}cos(\beta) & 0 & sin(\beta)\\ 0 & 1 & 0\\ -sin(\beta) & 0 & cos(\beta)\end{pmatrix}, Rz(\gamma) = \begin{pmatrix}cos(\gamma) & -sin(\gamma) & 0\\ sin(\gamma) & cos(\gamma) & 0\\ 0 & 0 & 1\end{pmatrix}
$$

Rotation matrices can be built with different sequences of 3 rotations about X, Y and Z axes. For instance proper Euler angles, ZXZ, or improper Euler (three different rotation axes) ZYX. Note that ZYX means, first Z, then Y then X.

Active rotations, i.e. rotations that move a physical object, are built right-multiplication, for instance for ZYX, $R=RxRyRz$, because Rz is applied first, then Ry, then Rx.
Passive rotations, i.e. rotations that define the rotation of a coordinate system with respect to another, is defined inversely. For instance, to apply ZYX order, $R=RzRyRx$.

In [ ]:
rz = (0 * np.pi) / 180.0  # yaw
ry = (15 * np.pi) / 180.0  # pitch
rx = (0 * np.pi) / 180.0  # roll
R_fv_wrt_lcs = utils.euler2R([rz, ry, rx], seq=utils.EulerSeq.ZYX)

The "pose" of the camera FV is then composed with the rotation and position matrices

In [ ]:
P_fv_wrt_lcs = utils.create_pose(R_fv_wrt_lcs, C_fv_wrt_lcs)
print(P_fv_wrt_lcs)

We can visualize the result

In [ ]:
# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_fv_wrt_lcs, 'fv', L=1)

ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

The same goes for other cameras. For instance, the rear view (RV) is equally pitched, but it also has yaw, so it points backward

In [ ]:
C_rv_wrt_lcs = np.array([[-0.5], [0.0], [0.5]])
rz = (180 * np.pi) / 180.0  # yaw
ry = (15 * np.pi) / 180.0  # pitch
rx = (0 * np.pi) / 180.0  # roll
R_rv_wrt_lcs = utils.euler2R([rz, ry, rx], seq=utils.EulerSeq.ZYX)
P_rv_wrt_lcs = utils.create_pose(R_rv_wrt_lcs, C_rv_wrt_lcs)
print(P_rv_wrt_lcs)

In [ ]:
# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_fv_wrt_lcs, 'fv', L=1)
plot_cs(P_rv_wrt_lcs, 'rv', L=1)

ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

### Transforms from Poses
Now, let's consider a point in 3D space, $\mathbf{X}$. In homogeneous coordinates it is a 4x1 vector.
If the point is expressed in the LCS coordinate system, $\mathbf{X}_{lcs}$, then we can also represent it with its coordinate values with respect to the cameras if we apply the transform using the pose.

Note that the Pose encodes the following transform:
$T_\text{fv_to_lcs} = P_\text{fv_to_lcs}$

Therefore, to transform FROM LCS to FV, we have to apply the inverse:
$$
T_\text{lcs_to_fv} = (P_\text{fv_to_lcs})^{-1}
$$

And therefore
$$
\mathbf{X}_\text{fv} = T_\text{lcs_to_fv} \mathbf{X}_\text{lcs}
$$

These equations hold true as long as the rotation $R_\text{fv_wrt_lcs}$ is created as a passive rotation, i.e. the rotation matrix is computed from Euler angles with right-handed signs for $R_x$, $R_y$, $R_z$ matrices, and the order of matrix multiplication is inverse to the intuitive order of angle rotation of the coordinate systems.
i.e. in this example, the RV pose is obtained by first applying 180º yaw (so it points backwards) and then 15º pitch (so it points to the ground). The rotation matrix is built as $R=R_z(\text{yaw})R_y(\text{pitch})R_x(\text{roll})$. Note that the order of matrix multiplication is $R_x$ first, then $R_y$, then $R_z$, exactly the inverse to the intuitive order.

Let's try it.
Let's create some points in front of the FV, at the ground:


In [ ]:
step_x = 1.0
step_y = 1.0
x_min = 5
x_max = 10
y_min = -3
y_max = 3
num_points_x = int((x_max - x_min)/step_x)
num_points_y = int((y_max - y_min)/step_y)
xm, ym, zm = utils.generate_grid(x_params=(x_min, x_max, num_points_x + 1), y_params=(y_min, y_max, num_points_y+1), z_params=(0, 0, 1))
points3d_4xN = utils.grid_as_4xN_points3d(xm, ym, zm)

In [ ]:
#Transform from LCS to FV:
T_lcs_to_fv = utils.inv(P_fv_wrt_lcs)
points3d_4xN_fv = T_lcs_to_fv @ points3d_4xN

In [ ]:
# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_fv_wrt_lcs, 'fv', L=1)
plot_cs(P_rv_wrt_lcs, 'rv', L=1)

# Plot the LCS points
ax.scatter(xm, ym, zm)

ax.set_xlim(0, 15)
ax.set_ylim(-6, 6)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

If we plot the points with respect to the FV, we can see the pitch angle affecting the view of the plane

In [ ]:
# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_fv_wrt_fv = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_fv_wrt_fv, 'fv', L=2)

# Plot3d the points
ax.plot3D(points3d_4xN_fv[0,:], points3d_4xN_fv[1,:], points3d_4xN_fv[2,:], '.r')

ax.set_xlim(0, 15)
ax.set_ylim(-6, 6)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')